In [ ]:

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.graphics.shapes import Drawing
from reportlab.graphics.charts.barcharts import VerticalBarChart
from io import BytesIO
from reportlab.graphics.shapes import Drawing, Line, String
import pyodbc
from reportlab.lib import fonts
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image
import pandas as pd
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
import schedule
import time
# Function to send email with PDF attachment
def send_email_with_pdf_attachment(pdf_content, receiver_email):
    # Email configuration
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    sender_email = 'nallagarivishnuvardhanreddy@gmail.com'
    sender_password = 'cchk mwpc vuml iazz'
    subject = 'PDF Attachment'

    # Create a multipart message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject

    # Attach PDF content
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(pdf_content.getvalue())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', 'attachment', filename="combined_tables.pdf")
    message.attach(part)

    # Connect to SMTP server and send email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())

    print(f'Email sent successfully to {receiver_email}')

# Establish connection to the SQL Server database
server = '10.0.1.71,4000'
database = 'HERITAGEBI'
username = 'HFLSQLReader'
password = 'HFL@12345'
driver = '{ODBC Driver 17 for SQL Server}'

conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

import pandas as pd

# Your existing code

# Execute the first SQL query
query1 = '''SELECT state, city, SalesOfficeID, salesoffice_Name, total_sales, tops
FROM (
    SELECT state, city, SalesOfficeID, salesoffice_Name, total_sales, 
           ROW_NUMBER() OVER (ORDER BY total_sales DESC) AS tops
    FROM (
        SELECT m.state, m.city, s.SalesOfficeID, m.plant_name AS salesoffice_Name, cast(SUM(s.salesquantity)as int) AS total_sales
        FROM dw.fSales s
        INNER JOIN dw.dSalesOfficeMaster m ON s.SalesOfficeID = m.PLANT
        WHERE s.BillingDate = CAST(DATEADD(day, -1, GETDATE()) AS DATE)
        GROUP BY m.state, m.city, s.SalesOfficeID, m.plant_name
    ) AS ranked_sales
) AS numbered_sales
WHERE tops <= 10
ORDER BY total_sales DESC;'''
cursor.execute(query1)
rows1 = cursor.fetchall()
df1 = pd.DataFrame([list(row) for row in rows1], columns=[desc[0] for desc in cursor.description])

# Execute the query to get average sales quantity
previousyear_query = '''SELECT 
    SalesOfficeID, 
    cast(SUM(salesquantity)as int) / 
    DATEDIFF(day, DATEADD(year, -1, EOMONTH(GETDATE(), -1)), DATEADD(year, -1, EOMONTH(GETDATE(), 0))) AS AverageSalesQuantityPerDay
FROM 
    dw.fSales 
WHERE 
    billingdate BETWEEN DATEADD(year, -1, DATEADD(day, 1, EOMONTH(GETDATE(), -1))) 
                     AND DATEADD(year, -1, EOMONTH(GETDATE(), 0)) 
  
GROUP BY 
    SalesOfficeID;'''
cursor.execute(previousyear_query)
avg_sales_rows = cursor.fetchall()
avg_sales_quantity = pd.DataFrame([list(row) for row in avg_sales_rows], columns=['SalesOfficeID', 'AverageSalesQuantity'])

# Merge the average sales quantity column to df1 based on SalesOfficeID
df1 = pd.merge(df1, avg_sales_quantity, on='SalesOfficeID', how='left')
df1['sales_difference'] = df1['total_sales'] - df1['AverageSalesQuantity']
df1['sales_percentage_difference'] = ((df1['sales_difference'] / df1['AverageSalesQuantity']) * 100).round(2).astype(str) + '%'


# Execute the second SQL query
query2 = '''SELECT state, city, SalesOfficeID, salesoffice_Name, total_sales, tops
FROM (
    SELECT state, city, SalesOfficeID, salesoffice_Name, total_sales, 
           ROW_NUMBER() OVER (ORDER BY total_sales ) AS tops
    FROM (
        SELECT m.state, m.city, s.SalesOfficeID, m.plant_name AS salesoffice_Name, cast(SUM(s.salesquantity)as int) AS total_sales
        FROM dw.fSales s
        INNER JOIN dw.dSalesOfficeMaster m ON s.SalesOfficeID = m.PLANT
        WHERE s.BillingDate = CAST(DATEADD(day, -1, GETDATE()) AS DATE)
        GROUP BY m.state, m.city, s.SalesOfficeID, m.plant_name
    ) AS ranked_sales
) AS numbered_sales
WHERE tops <= 10
ORDER BY total_sales desc;'''

cursor.execute(query2)
rows2 = cursor.fetchall()
df2 = pd.DataFrame([list(row) for row in rows2], columns=[desc[0] for desc in cursor.description])
cursor.execute(previousyear_query)
avg_sales_rows = cursor.fetchall()
avg_sales_quantity = pd.DataFrame([list(row) for row in avg_sales_rows], columns=['SalesOfficeID', 'AverageSalesQuantity'])

# Merge the average sales quantity column to df1 based on SalesOfficeID
df2 = pd.merge(df2, avg_sales_quantity, on='SalesOfficeID', how='left')

# Calculate the difference between total_sales and average sales quantity
df2['sales_difference'] = df2['total_sales'] - df2['AverageSalesQuantity']
df2['sales_percentage_difference'] = ((df2['sales_difference'] / df2['AverageSalesQuantity']) * 100).round(2).astype(str) + '%'

cursor.close()

conn.close()

# Specify the output file path for the PDF
output_file = "combined_tables.pdf"

custom_page_size = (1000,800)  # Width x Height in points

# Create a PDF document
pdf_buffer = BytesIO()
pdf = SimpleDocTemplate(pdf_buffer, pagesize=custom_page_size)

# Create styles for paragraphs
styles = getSampleStyleSheet()
title_style = styles['Heading1']
normal_style = styles['Normal']

# Define data for tables
data1 = [df1.columns.values.tolist()] + df1.values.tolist()
data2 = [df2.columns.values.tolist()] + df2.values.tolist()

# Add the tables and charts to the PDF content
content = []
image_path = "C:\\Users\\HP\\Downloads\\heritage.cms"
image = Image(image_path)
content.append(image)
title_style = getSampleStyleSheet()["Heading1"]
title_style.fontSize = 18  # Adjust font size as needed
title_style.textColor = colors.black  # Change color to black or any other color you prefer

# Add the text with the modified style
content.append(Paragraph("HERITAGE FOODs Pvt.Ltd", title_style))
# Add title and table 1
content.append(Paragraph("Top 10 Sales Report:", title_style))
table1 = Table(data1)
table1.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                            ('GRID', (0, 0), (-1, -1), 1, colors.black)]))



top_chart_data = sorted(df1['tops'].tolist(), reverse=True)
top_category_names =df1['SalesOfficeID'].astype(str).tolist()
top_drawing = Drawing(400, 200)
top_bc = VerticalBarChart()
top_bc.x = 50
top_bc.y = 40
top_bc.height = 150
top_bc.width = 300
top_bc.data = [top_chart_data]
top_bc.categoryAxis.labels.angle =25
top_bc.categoryAxis.labels.fontSize = 12
top_bc.categoryAxis.categoryNames = top_category_names
top_bc.bars[0].fillColor = colors.green

for i, label in enumerate(top_category_names):
    top_bc.bars[i].name = label

top_drawing.add(top_bc)
content.append(table1)
content.append(top_drawing)

content.append(Paragraph("Bottom 10 Sales Report:", title_style))
table2 = Table(data2)
table2.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                            ('GRID', (0, 0), (-1, -1), 1, colors.black)]))


bottom_chart_data = sorted(df2['tops'].tolist(), reverse=False)  # Use total_sales column for chart data
bottom_category_names = sorted(df2['SalesOfficeID'].astype(str).tolist(),reverse=True)

bottom_drawing = Drawing(400, 200)
bottom_bc = VerticalBarChart()
bottom_bc.x = 50
bottom_bc.y = 40
bottom_bc.height = 150
bottom_bc.width = 300
bottom_bc.data = [bottom_chart_data]
bottom_bc.categoryAxis.labels.angle = 25
bottom_bc.categoryAxis.labels.fontSize = 12
bottom_bc.categoryAxis.categoryNames = bottom_category_names
bottom_bc.bars[0].fillColor = colors.blue

# Add the bars to the drawing
bottom_drawing.add(bottom_bc)

content.append(table2)
content.append(bottom_drawing)

pdf.build(content)

# Rewind the buffer
pdf_buffer.seek(0)
def job():
    print("Sending email...")
    # Specify recipient's email address
    receiver_email = 'vnallagari1919@gmail.com'
    # Call function to send email with PDF attachment
    send_email_with_pdf_attachment(pdf_buffer, receiver_email)

# Schedule the job to run every 1 minute
#schedule.every(1).minutes.do(job)
# schedule.every().day.at("08:00").do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)
# Send email with PDF attachment
'''receiver_email = 'vnallagari1919@gmail.com'  # Update with recipient's email address
send_email_with_pdf_attachment(pdf_buffer, receiver_email)

print(f"Data exported to PDF and sent via email to {receiver_email}")'''
